In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
stp_wrds= stopwords.words('english')

In [2]:
df_train= pd.read_csv('hm_train.csv')
df_test= pd.read_csv('hm_test.csv')

In [3]:
df_train.head()

,hmid,reflection_period,cleaned_hm,num_sentence,predicted_category
0,27673,24h,I went on a successful date with someone I fel...,1,affection
1,27674,24h,I was happy when my son got 90% marks in his e...,1,affection
2,27675,24h,I went to the gym this morning and did yoga.,1,exercise
3,27676,24h,We had a serious talk with some friends of our...,2,bonding
4,27677,24h,I went with grandchildren to butterfly display...,1,affection


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60321 entries, 0 to 60320
Data columns (total 5 columns):
hmid                  60321 non-null int64
reflection_period     60321 non-null object
cleaned_hm            60321 non-null object
num_sentence          60321 non-null int64
predicted_category    60321 non-null object
dtypes: int64(2), object(3)
memory usage: 2.3+ MB


In [6]:
df_test.head()

,hmid,reflection_period,cleaned_hm,num_sentence
0,88305,3m,I spent the weekend in Chicago with my friends.,1
1,88306,3m,We moved back into our house after a remodel. ...,2
2,88307,3m,My fiance proposed to me in front of my family...,1
3,88308,3m,I ate lobster at a fancy restaurant with some ...,1
4,88309,3m,I went out to a nice restaurant on a date with...,5


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40213 entries, 0 to 40212
Data columns (total 4 columns):
hmid                 40213 non-null int64
reflection_period    40213 non-null object
cleaned_hm           40213 non-null object
num_sentence         40213 non-null int64
dtypes: int64(2), object(2)
memory usage: 1.2+ MB


In [3]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stp_wrds])
    return sen_new
wo_stpwrds = [remove_stopwords(r.split()) for r in df_train['cleaned_hm']]

In [4]:
wo_stp = pd.DataFrame()
wo_stp['cleaned_hm'] = wo_stpwrds
wo_stp['predicted_category'] = df_train['predicted_category']

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(wo_stp['cleaned_hm'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [6]:
from sklearn.svm import LinearSVC
svc= LinearSVC().fit(X_train_tfidf, wo_stp['predicted_category'])


In [31]:
X_train_tfidf.shape

(60321, 20507)

In [7]:
g= svc.predict(count_vect.transform(df_test['cleaned_hm']))

In [9]:
g.shape

(40213,)

In [10]:
result = pd.DataFrame()

In [11]:
result['hmid'] = df_test['hmid']
result['predicted_category'] = g

In [12]:
result.head()

,hmid,predicted_category
0,88305,bonding
1,88306,affection
2,88307,affection
3,88308,bonding
4,88309,affection


In [14]:
result.to_csv('result_lsvm_wo.csv')